## 2. Constraint Satisfaction Problems

First, we wanted to add every bit of information we got from the assingment and create a easy representation for us. So we can later use them in our constraints easily and declaratively, removing the need for any magic numbers/variables.

We also created a distinct name for each group from each major, so we can later identify and check that we are not missing out or assigning the same group multiple times either than needed or at the same time.


#### Define Static Variables

In [3]:
# Define all static variables from the assignment description
rooms = ["L1","L2", "L3"]
time_slots = ["8-9","10-11","13-14","15-16"]
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
num_presentations_needed = 3
num_max_commitee_room_changes_per_day = 2

#### Load Data from CSV

In [4]:
from csv import DictReader
def load_data_from_csv(filepath):
    """Load major data from CSV file"""
    courses = {}

    with open(filepath, 'r', encoding='utf-8') as file:
        reader = DictReader(file, delimiter=';')

        for row in reader:
            major = row['Studiengang']
            courses[major] = {
                'num_groups': int(row['Projektgruppen']),
                'num_committees': int(row['Kommissionen']),
                'groups': [f'{major}-G{i+1}' for i in range(int(row['Projektgruppen']))],
                'committees': [f'{major}-C{i+1}' for i in range(int(row['Kommissionen']))],
            }

    total_presentations = sum(course['num_groups'] * 3 for course in courses.values())
    max_possible_presentations = len(rooms)*len(time_slots)*len(days)
    print(f'{total_presentations}/{max_possible_presentations}')
    return courses

In [5]:
from constraint import Problem, AllDifferentConstraint

def solve(filepath):
    courses = load_data_from_csv(filepath)
    print()

    problem = Problem()
    n = 0

    for course, data in courses.items():
        for group in data['groups']:
            for p in range(1, num_presentations_needed + 1):
                var_name = f"{group}-P{p}"
                problem.addVariable(f'{var_name}-day', days)
                problem.addVariable(f'{var_name}-room', rooms)
                problem.addVariable(f"{var_name}-slot", time_slots)
                problem.addVariable(f"{var_name}-committee", data['committees'])
                n+=4

    print(f'Added {n} variables')

solve("DS_CSP_1/pr_conf_001.csv")

60/60

Added 240 variables


In [19]:
courses = load_data_from_csv("DS_CSP_1/pr_conf_001.csv")

problem = Problem()

day_domain = days
slot_domain = time_slots
room_domain = rooms

for course, data in courses.items():
    for group in data['groups']:
        for p in range(1, num_presentations_needed + 1):
            var_name = f"{group}-P{p}"
            problem.addVariable(f'{var_name}-day', day_domain)
            problem.addVariable(f'{var_name}-room', room_domain)
            problem.addVariable(f"{var_name}-slot", slot_domain)
            problem.addVariable(f"{var_name}-committee", data['committees'])



DEBUG MODUS - Constraint Analyse
Raum-Konflikte: True
Gruppen-Konflikte: False
Tag-Übergänge: False
Slot-Ordnung: False
Gruppen-Ordnung: False
✓ Variablen: 60 Prüfungsslots
✓ Gruppen: 20
✓ Verfügbare Raum-Zeit-Slots: 60

⏳ Füge Raum-Constraints hinzu...
✓ 1770 Raum-Constraints

🔍 Starte Solver...
